Część 1: Naive Bayes (Spam i Grypa)
Krok 1: Zadanie "Email Spam" – Obliczenia ręczne
Wklej ten kod do pierwszej komórki. Implementujemy tu wzór Bayesa „od zera”, aby zobaczyć, jak powstaje wynik.

In [2]:
import pandas as pd
import numpy as np

data_spam = {
    'Słowo_1': ['darmowy', 'darmowy', 'spotkanie', 'raport', 'oferta', 'darmowy', 'spotkanie', 'oferta', 'raport', 'darmowy', 'spotkanie', 'oferta'],
    'Słowo_2': ['wygrana', 'wygrana', 'jutro', 'kwartalny', 'specjalna', 'rabat', 'dziś', 'limitowana', 'miesięczny', 'rabat', 'pilne', 'wyjątkowa'],
    'Wykrzyknik': ['TAK', 'TAK', 'NIE', 'NIE', 'TAK', 'TAK', 'NIE', 'TAK', 'NIE', 'TAK', 'NIE', 'TAK'],
    'Spam': ['TAK', 'TAK', 'NIE', 'NIE', 'NIE', 'TAK', 'NIE', 'NIE', 'NIE', 'TAK', 'NIE', 'NIE']
}
df_spam = pd.DataFrame(data_spam)

total = len(df_spam)
p_spam_tak = len(df_spam[df_spam['Spam'] == 'TAK']) / total
p_spam_nie = len(df_spam[df_spam['Spam'] == 'NIE']) / total

nowy_email = {'Słowo_1': 'darmowy', 'Słowo_2': 'wygrana', 'Wykrzyknik': 'TAK'}

prob_tak = 1.0
subset_tak = df_spam[df_spam['Spam'] == 'TAK']
for cecha, wartosc in nowy_email.items():
    prob_tak *= len(subset_tak[subset_tak[cecha] == wartosc]) / len(subset_tak)

prob_nie = 1.0
subset_nie = df_spam[df_spam['Spam'] == 'NIE']
for cecha, wartosc in nowy_email.items():
    prob_nie *= len(subset_nie[subset_nie[cecha] == wartosc]) / len(subset_nie)

posterior_tak = p_spam_tak * prob_tak
posterior_nie = p_spam_nie * prob_nie

norm = posterior_tak + posterior_nie
wynik_tak = posterior_tak / norm if norm > 0 else 0
wynik_nie = posterior_nie / norm if norm > 0 else 0

print(f"Prawdopodobieństwo SPAM = TAK: {wynik_tak:.4f}")
print(f"Prawdopodobieństwo SPAM = NIE: {wynik_nie:.4f}")

Prawdopodobieństwo SPAM = TAK: 1.0000
Prawdopodobieństwo SPAM = NIE: 0.0000


Krok 2: Zadanie "Email Spam" – Biblioteka Scikit-Learn
Teraz użyjemy gotowego modelu, który stosuje "wygładzanie" (smoothing), żeby uniknąć problemu mnożenia przez zero.

In [3]:
from sklearn.naive_bayes import CategoricalNB
from sklearn.preprocessing import OrdinalEncoder

enc = OrdinalEncoder()
X = enc.fit_transform(df_spam.drop('Spam', axis=1))
y = df_spam['Spam'].map({'TAK': 1, 'NIE': 0})

model = CategoricalNB(alpha=1.0)
model.fit(X, y)

test_vector = pd.DataFrame([['darmowy', 'wygrana', 'TAK']], columns=['Słowo_1', 'Słowo_2', 'Wykrzyknik'])
test_vector_enc = enc.transform(test_vector)

proba = model.predict_proba(test_vector_enc)
print(f"Sklearn P(Spam=TAK): {proba[0][1]:.4f}")

Sklearn P(Spam=TAK): 0.9679


Krok 3: Zadanie "Diagnoza Grypy"
Analizujemy dane medyczne i przewidujemy chorobę dla 3 pacjentów.

In [4]:
data_grypa = {
    'Gorączka': ['wysoka', 'wysoka', 'niska', 'wysoka', 'niska', 'wysoka', 'niska', 'wysoka', 'umiarkowana', 'wysoka', 'niska', 'umiarkowana', 'wysoka', 'niska', 'umiarkowana'],
    'Kaszel': ['TAK', 'TAK', 'NIE', 'TAK', 'NIE', 'TAK', 'NIE', 'TAK', 'TAK', 'NIE', 'NIE', 'TAK', 'TAK', 'NIE', 'TAK'],
    'Zmęczenie': ['duże', 'duże', 'małe', 'duże', 'małe', 'duże', 'małe', 'umiarkowane', 'duże', 'małe', 'małe', 'umiarkowane', 'duże', 'małe', 'duże'],
    'Grypa': ['TAK', 'TAK', 'NIE', 'TAK', 'NIE', 'TAK', 'NIE', 'TAK', 'TAK', 'NIE', 'NIE', 'NIE', 'TAK', 'NIE', 'TAK']
}
df_grypa = pd.DataFrame(data_grypa)

print(df_grypa['Grypa'].value_counts())

pacjenci = [
    {'Gorączka': 'wysoka', 'Kaszel': 'TAK', 'Zmęczenie': 'duże'},
    {'Gorączka': 'niska', 'Kaszel': 'NIE', 'Zmęczenie': 'małe'},
    {'Gorączka': 'umiarkowana', 'Kaszel': 'TAK', 'Zmęczenie': 'umiarkowane'}
]

p_grypa_tak = len(df_grypa[df_grypa['Grypa'] == 'TAK']) / len(df_grypa)
p_grypa_nie = len(df_grypa[df_grypa['Grypa'] == 'NIE']) / len(df_grypa)

for i, pacjent in enumerate(pacjenci):
    prob_tak = 1.0
    subset_tak = df_grypa[df_grypa['Grypa'] == 'TAK']
    for k, v in pacjent.items():
        prob_tak *= len(subset_tak[subset_tak[k] == v]) / len(subset_tak)

    prob_nie = 1.0
    subset_nie = df_grypa[df_grypa['Grypa'] == 'NIE']
    for k, v in pacjent.items():
        prob_nie *= len(subset_nie[subset_nie[k] == v]) / len(subset_nie)

    post_tak = p_grypa_tak * prob_tak
    post_nie = p_grypa_nie * prob_nie
    wynik = post_tak / (post_tak + post_nie)

    print(f"Pacjent {chr(65+i)}: P(Grypa=TAK) = {wynik:.4f}")

Grypa
TAK    8
NIE    7
Name: count, dtype: int64
Pacjent A: P(Grypa=TAK) = 1.0000
Pacjent B: P(Grypa=TAK) = 0.0000
Pacjent C: P(Grypa=TAK) = 0.9245


Część 2: Zadanie Titanic (Regresja Logistyczna)
Krok 4: Wczytanie i przygotowanie danych
Wczytujemy zbiór Titanic, usuwamy zbędne kolumny (np. 'deck' ma za dużo braków), uzupełniamy wiek medianą i zamieniamy tekst na liczby (One-Hot Encoding).

In [5]:
import seaborn as sns
from sklearn.model_selection import train_test_split

df = sns.load_dataset('titanic')

df_clean = df.drop(columns=['deck', 'embark_town', 'alive', 'who', 'adult_male'])
df_clean['age'] = df_clean['age'].fillna(df_clean['age'].median())
df_clean = df_clean.dropna(subset=['embarked'])

df_encoded = pd.get_dummies(df_clean, columns=['sex', 'embarked', 'class'], drop_first=True)

X = df_encoded.drop('survived', axis=1)
y = df_encoded['survived']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

print(f"Rozmiar zbioru treningowego: {X_train.shape}")
print(f"Rozmiar zbioru testowego: {X_test.shape}")

Rozmiar zbioru treningowego: (711, 11)
Rozmiar zbioru testowego: (178, 11)


Krok 5: Budowa modelu i analiza współczynników
Trenujemy regresję logistyczną i sprawdzamy, które cechy najbardziej wpływają na przeżycie.

In [6]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

coefs = pd.DataFrame({'cecha': X.columns, 'wspolczynnik': model.coef_[0]})
coefs['wplyw_bezwzgledny'] = coefs['wspolczynnik'].abs()
coefs = coefs.sort_values('wplyw_bezwzgledny', ascending=False)

print("Top 3 najważniejsze cechy:")
print(coefs.head(3))

coefs['odds_ratio'] = np.exp(coefs['wspolczynnik'])
print("\nIloraz szans (Odds Ratio) dla płci męskiej:")
print(coefs[coefs['cecha'] == 'sex_male'][['cecha', 'odds_ratio']])

Top 3 najważniejsze cechy:
      cecha  wspolczynnik  wplyw_bezwzgledny
6  sex_male     -2.475732           2.475732
0    pclass     -0.783356           0.783356
5     alone     -0.624589           0.624589

Iloraz szans (Odds Ratio) dla płci męskiej:
      cecha  odds_ratio
6  sex_male    0.084101


Krok 6: Ewaluacja (Ocena jakości)
Sprawdzamy skuteczność modelu na danych testowych.

In [7]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score

y_pred = model.predict(X_test)
y_proba = model.predict_proba(X_test)[:, 1]

cm = confusion_matrix(y_test, y_pred)
tn, fp, fn, tp = cm.ravel()
specificity = tn / (tn + fp)

print(f"Accuracy (Dokładność): {accuracy_score(y_test, y_pred):.4f}")
print(f"Precision (Precyzja): {precision_score(y_test, y_pred):.4f}")
print(f"Recall (Czułość): {recall_score(y_test, y_pred):.4f}")
print(f"Specificity (Swoistość): {specificity:.4f}")
print(f"AUC: {roc_auc_score(y_test, y_proba):.4f}")

Accuracy (Dokładność): 0.8258
Precision (Precyzja): 0.8246
Recall (Czułość): 0.6912
Specificity (Swoistość): 0.9091
AUC: 0.8650


Krok 7: Eksperymenty z regularyzacją
Sprawdzamy, jak parametry modelu wpływają na jego jakość (Zadanie 2.5 i 2.6).

In [8]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

c_values = [0.001, 0.01, 0.1, 1, 10, 100]
print("Wpływ parametru C na AUC:")

for c in c_values:
    lr = LogisticRegression(C=c, max_iter=1000)
    lr.fit(X_train_scaled, y_train)
    auc = roc_auc_score(y_test, lr.predict_proba(X_test_scaled)[:, 1])
    print(f"C={c}: AUC={auc:.4f}")

Wpływ parametru C na AUC:
C=0.001: AUC=0.8311
C=0.01: AUC=0.8497
C=0.1: AUC=0.8606
C=1: AUC=0.8646
C=10: AUC=0.8647
C=100: AUC=0.8648
